# 2025 DL Lab8: RL Assignment_Super Mario World

**Your Answer:**    
Hi I'm XXX, XXXXXXXXXX.

## Overview
This project implements a **Deep Reinforcement Learning** pipeline to train an autonomous agent for Super Mario World. Leveraging the **Proximal Policy Optimization (PPO)** algorithm, the system interacts with the **stable-retro** environment to master the YoshiIsland1 level. Key components include a custom Vision Backbone for extracting features from raw pixel data and a suite of Environment Wrappers that handle frame preprocessing, action discretization, and reward shaping to facilitate efficient learning.

Reward function implement  
should do something in the beginning (monster attack)  
Custom PPO implement  
pre train weight 差不多，主要是 reward function  
model weight capacity 1GB  
class name 不要動 (可以新增，但是原本有的不要動)

## Imports

In [1]:
import os
import numpy as np
import retro
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.vec_env import VecNormalize

from eval import evaluate_policy, record_video
from custom_policy import VisionBackbonePolicy, CustomPPO

/home/m314834001/Lab8/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Configuration

In [2]:
# Game Settings
GAME = "SuperMarioWorld-Snes"
STATE = "YoshiIsland1"

# Training Settings
# TOTAL_STEPS = 0x1400000 # 20,971,520
# TOTAL_STEPS = 0x0A00000 # 10,485,760
TOTAL_STEPS = 0x1900000
TRAIN_CHUNK = 0x0040000 #    262,144
N_ENVS = 16

# Evaluation & Recording Settings
EVAL_EPISODES = 3
EVAL_MAX_STEPS = 18000
RECORD_STEPS = 1800

# Directories
LOG_DIR = "./runs_smw"
VIDEO_DIR       = os.path.join(LOG_DIR, "videos")
CKPT_DIR        = os.path.join(LOG_DIR, "checkpoints")
TENSORBOARD_LOG = os.path.join(LOG_DIR, "tb")

os.makedirs(LOG_DIR,   exist_ok=True)
os.makedirs(CKPT_DIR,  exist_ok=True)
os.makedirs(VIDEO_DIR, exist_ok=True)

## Environment Functions

In [3]:
from wrappers import make_base_env
def _make_env_thunk(game: str, state: str):
    """Return a function that creates an environment (for multiprocessing)."""
    def _thunk():
        return make_base_env(game, state)
    return _thunk

def make_vec_env(game: str, state: str, n_envs: int, use_subproc: bool = True):
    """Create a vectorized environment (multiple envs running in parallel)."""
    env_fns = [_make_env_thunk(game, state) for _ in range(n_envs)]
    
    if use_subproc and n_envs > 1:
        vec_env = SubprocVecEnv(env_fns)
    else:
        vec_env = DummyVecEnv(env_fns)

    return vec_env

## Initialize Env & Model

In [4]:
# 1. Create Training Environment
train_env = make_vec_env(GAME, STATE, n_envs=N_ENVS)
# train_env = VecNormalize(train_env, norm_obs=True, norm_reward=True, clip_obs=10., clip_reward=10.)
print(f"Environment created: {GAME} - {STATE} with {N_ENVS} parallel envs.")

checkpoint_path = "None" # 6225920 (19) 有破壞
# checkpoint_path = "runs_smw/checkpoints/Enc5_45.zip"

best_mean = -1e18
trained = 0
round_idx = 0

# 2. Initialize Model
if os.path.exists(checkpoint_path):
    # 讀取現有模型
    model = CustomPPO.load(
        checkpoint_path, 
        env=train_env,
        device="cuda:0" # 確保使用 GPU
    )
    trained = model.num_timesteps
    round_idx = int(trained / TRAIN_CHUNK)
    print(f"[Sucess] Loaded model from {checkpoint_path}")
    print(f"trained: {trained}, round_index: {round_idx}")
else:
    print(f"[Fail] Can't load {checkpoint_path}. Will use new model")
    model = CustomPPO(
        VisionBackbonePolicy,
        train_env,
        policy_kwargs   = dict(normalize_images=False),
        n_epochs        = 4,
        n_steps         = 512,
        batch_size      = 512,
        learning_rate   = 1e-4,
        verbose         = 1,
        gamma           = 0.96875,
        kl_coef         = 1,
        clip_range      = 0.125,
        ent_coef        = 0.0375,
        tensorboard_log = TENSORBOARD_LOG,
    )

Environment created: SuperMarioWorld-Snes - YoshiIsland1 with 16 parallel envs.
[Fail] Can't load None. Will use new model
Using cuda:0 device


In [5]:
# model.save("policy")

In [6]:
import os
import glob
from custom_policy import CustomPPO
from eval import record_video  # 確保 eval.py 在同一目錄下

# ================= 設定區 =================
target_numbers = [59, 63, 67]

# ================= 執行迴圈 =================
print(f"準備測試以下 Checkpoints: {target_numbers}")

for num in target_numbers:
    model_path = os.path.join(CKPT_DIR, f"Enc5_{num}.zip")
    
    # 檢查檔案是否存在
    if not os.path.exists(model_path):
        print(f"⚠️ 找不到檔案: {model_path}，跳過。")
        continue
    
    print(f"\n[{num}] 正在載入模型: {model_path} ...")
    
    try:
        # 1. 載入模型 (不需要 env 參數也能載入權重)
        # 如果你有改過 CustomPPO 的參數，load 會自動讀取 zip 裡的設定
        model = CustomPPO.load(model_path, device="auto") # device="auto" 會自動用 GPU
        
        # 2. 錄製影片
        prefix_name = f"test_{num}"
        print(f"[{num}] 正在錄影 (長度 {RECORD_STEPS} steps)...")
        
        record_video(
            model=model,
            game=GAME,
            state=STATE,
            out_dir=VIDEO_DIR,
            video_len=RECORD_STEPS,
            prefix=prefix_name
        )
        print(f"✅ 完成！影片已儲存為 {prefix_name}.mp4")
        
    except Exception as e:
        print(f"❌ 發生錯誤 (Model: {num}): {e}")

print("\n所有測試結束。")

準備測試以下 Checkpoints: [59, 63, 67]

[59] 正在載入模型: ./runs_smw/checkpoints/Enc5_59.zip ...
[59] 正在錄影 (長度 1800 steps)...
Saved video to ./runs_smw/videos/test_59.mp4
✅ 完成！影片已儲存為 test_59.mp4

[63] 正在載入模型: ./runs_smw/checkpoints/Enc5_63.zip ...
[63] 正在錄影 (長度 1800 steps)...
Saved video to ./runs_smw/videos/test_63.mp4
✅ 完成！影片已儲存為 test_63.mp4

[67] 正在載入模型: ./runs_smw/checkpoints/Enc5_67.zip ...
[67] 正在錄影 (長度 1800 steps)...
Saved video to ./runs_smw/videos/test_67.mp4
✅ 完成！影片已儲存為 test_67.mp4

所有測試結束。


## Training Loop

In [ ]:
try:
    while trained < TOTAL_STEPS:
        round_idx += 1
        chunk = min(TRAIN_CHUNK, TOTAL_STEPS - trained)
        # chunk = 2000
        label = "Enc5"
        tagged_label = f"{label}_{int(trained/TRAIN_CHUNK)}"

        print(f"\n=== Round {round_idx} | Learn {chunk} steps (Total trained: {trained}) ===")
        
        # --- Train ---
        model.learn(total_timesteps=chunk, reset_num_timesteps=False, tb_log_name=label)
        trained += chunk

        # --- Save Checkpoint ---
        ckpt_path = os.path.join(CKPT_DIR, f"{tagged_label}.zip")
        model.save(ckpt_path)
        print(f"Saved checkpoint: {ckpt_path}")

        # --- Evaluate ---
        mean_ret, best_ret = evaluate_policy(
            model,
            GAME,
            STATE,
            n_episodes=EVAL_EPISODES,
            max_steps=EVAL_MAX_STEPS,
        )
        print(f"[EVAL] Mean Return: {mean_ret:.3f}, Best Return: {best_ret:.3f}")

        # --- Save Best Model ---
        # if mean_ret > best_mean:
        #     best_mean = mean_ret
        #     best_path = os.path.join(LOG_DIR, "best_model.zip")
        #     model.save(best_path)
        #     print(f"New best record. Saved to {best_path}")

        # --- Record Video ---
        out_path = os.path.join(VIDEO_DIR, label)
        os.makedirs(out_path,  exist_ok=True)
        record_video(
            model,
            GAME,
            STATE,
            VIDEO_DIR,
            video_len=RECORD_STEPS,
            prefix=f"{label}/{tagged_label}_{mean_ret:.2f}",
        )

except KeyboardInterrupt:
    print("\nTraining interrupted manually.")

finally:
    train_env.close()
    print("Training finished. Environment closed.")
    
"""
tensorboard --logdir=./runs_smw/tb
"""


=== Round 47 | Learn 262144 steps (Total trained: 12058624) ===
Logging to ./runs_smw/tb/Enc5_0


/home/m314834001/Lab8/.venv/lib/python3.12/site-packages/rich/live.py:256: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

---------------------------------
| time/              |          |
|    fps             | 1104     |
|    iterations      | 1        |
|    time_elapsed    | 7        |
|    total_timesteps | 12066816 |
---------------------------------
--------------------------------------
| time/                 |            |
|    fps                | 870        |
|    iterations         | 2          |
|    time_elapsed       | 18         |
|    total_timesteps    | 12075008   |
| train/                |            |
|    approx_kl          | 0.0114237  |
|    entropy_loss       | -1.75      |
|    explained_variance | 0.88       |
|    learning_rate      | 0.0001     |
|    loss               | 0.213      |
|    mean_step_reward   | 0.21493906 |
|    n_updates          | 5892       |
|    policyGradLoss     | 0.00225    |
|    value_loss         | 1.18       |
--------------------------------------
---------------------------------------
| time/                 |             |
|    fps           

Saved checkpoint: ./runs_smw/checkpoints/Enc5_46.zip
[EVAL] Mean Return: 29.241, Best Return: 29.907
Saved video to ./runs_smw/videos/Enc5/Enc5_46_29.24.mp4

=== Round 48 | Learn 262144 steps (Total trained: 12320768) ===
Logging to ./runs_smw/tb/Enc5_0
---------------------------------
| time/              |          |
|    fps             | 1167     |
|    iterations      | 1        |
|    time_elapsed    | 7        |
|    total_timesteps | 12328960 |
---------------------------------
---------------------------------------
| time/                 |             |
|    fps                | 904         |
|    iterations         | 2           |
|    time_elapsed       | 18          |
|    total_timesteps    | 12337152    |
| train/                |             |
|    approx_kl          | 0.010246567 |
|    entropy_loss       | -1.74       |
|    explained_variance | 0.952       |
|    learning_rate      | 0.0001      |
|    loss               | 0.874       |
|    mean_step_reward   | 0.

Saved checkpoint: ./runs_smw/checkpoints/Enc5_47.zip
[EVAL] Mean Return: 338.340, Best Return: 344.340
Saved video to ./runs_smw/videos/Enc5/Enc5_47_338.34.mp4

=== Round 49 | Learn 262144 steps (Total trained: 12582912) ===
Logging to ./runs_smw/tb/Enc5_0
---------------------------------
| time/              |          |
|    fps             | 1086     |
|    iterations      | 1        |
|    time_elapsed    | 7        |
|    total_timesteps | 12591104 |
---------------------------------
---------------------------------------
| time/                 |             |
|    fps                | 938         |
|    iterations         | 2           |
|    time_elapsed       | 17          |
|    total_timesteps    | 12599296    |
| train/                |             |
|    approx_kl          | 0.011333286 |
|    entropy_loss       | -1.78       |
|    explained_variance | 0.946       |
|    learning_rate      | 0.0001      |
|    loss               | 0.568       |
|    mean_step_reward   |

...

Saved checkpoint: ./runs_smw/checkpoints/Enc5_59.zip
[EVAL] Mean Return: 558.763, Best Return: 564.097
Saved video to ./runs_smw/videos/Enc5/Enc5_59_558.76.mp4

=== Round 61 | Learn 262144 steps (Total trained: 15728640) ===
Logging to ./runs_smw/tb/Enc5_0
---------------------------------
| time/              |          |
|    fps             | 1079     |
|    iterations      | 1        |
|    time_elapsed    | 7        |
|    total_timesteps | 15736832 |
---------------------------------
----------------------------------------
| time/                 |              |
|    fps                | 873          |
|    iterations         | 2            |
|    time_elapsed       | 18           |
|    total_timesteps    | 15745024     |
| train/                |              |
|    approx_kl          | 0.0091503905 |
|    entropy_loss       | -1.7         |
|    explained_variance | 0.962        |
|    learning_rate      | 0.0001       |
|    loss               | 1.61         |
|    mean_ste

Saved checkpoint: ./runs_smw/checkpoints/Enc5_60.zip
[EVAL] Mean Return: 73.438, Best Return: 73.438
Saved video to ./runs_smw/videos/Enc5/Enc5_60_73.44.mp4

=== Round 62 | Learn 262144 steps (Total trained: 15990784) ===
Logging to ./runs_smw/tb/Enc5_0
---------------------------------
| time/              |          |
|    fps             | 1272     |
|    iterations      | 1        |
|    time_elapsed    | 6        |
|    total_timesteps | 15998976 |
---------------------------------
---------------------------------------
| time/                 |             |
|    fps                | 921         |
|    iterations         | 2           |
|    time_elapsed       | 17          |
|    total_timesteps    | 16007168    |
| train/                |             |
|    approx_kl          | 0.008927664 |
|    entropy_loss       | -1.76       |
|    explained_variance | 0.973       |
|    learning_rate      | 0.0001      |
|    loss               | 1.44        |
|    mean_step_reward   | 0.

Saved checkpoint: ./runs_smw/checkpoints/Enc5_61.zip
[EVAL] Mean Return: 95.840, Best Return: 96.507
Saved video to ./runs_smw/videos/Enc5/Enc5_61_95.84.mp4

=== Round 63 | Learn 262144 steps (Total trained: 16252928) ===
Logging to ./runs_smw/tb/Enc5_0
---------------------------------
| time/              |          |
|    fps             | 1097     |
|    iterations      | 1        |
|    time_elapsed    | 7        |
|    total_timesteps | 16261120 |
---------------------------------
---------------------------------------
| time/                 |             |
|    fps                | 947         |
|    iterations         | 2           |
|    time_elapsed       | 17          |
|    total_timesteps    | 16269312    |
| train/                |             |
|    approx_kl          | 0.010022578 |
|    entropy_loss       | -1.71       |
|    explained_variance | 0.972       |
|    learning_rate      | 0.0001      |
|    loss               | 0.716       |
|    mean_step_reward   | 0.

Saved checkpoint: ./runs_smw/checkpoints/Enc5_62.zip
[EVAL] Mean Return: 49.019, Best Return: 49.686
Saved video to ./runs_smw/videos/Enc5/Enc5_62_49.02.mp4

=== Round 64 | Learn 262144 steps (Total trained: 16515072) ===
Logging to ./runs_smw/tb/Enc5_0
---------------------------------
| time/              |          |
|    fps             | 1105     |
|    iterations      | 1        |
|    time_elapsed    | 7        |
|    total_timesteps | 16523264 |
---------------------------------
---------------------------------------
| time/                 |             |
|    fps                | 903         |
|    iterations         | 2           |
|    time_elapsed       | 18          |
|    total_timesteps    | 16531456    |
| train/                |             |
|    approx_kl          | 0.011155065 |
|    entropy_loss       | -1.69       |
|    explained_variance | 0.978       |
|    learning_rate      | 0.0001      |
|    loss               | 0.372       |
|    mean_step_reward   | 0.

Saved checkpoint: ./runs_smw/checkpoints/Enc5_63.zip
[EVAL] Mean Return: 554.997, Best Return: 562.997
Saved video to ./runs_smw/videos/Enc5/Enc5_63_555.00.mp4

=== Round 65 | Learn 262144 steps (Total trained: 16777216) ===
Logging to ./runs_smw/tb/Enc5_0


## Display Video

In [1]:
from IPython.display import Video
import glob
label = "Dec22A"

list_of_files = glob.glob(os.path.join(VIDEO_DIR, label, '*.mp4')) 
if list_of_files:
    latest_file = max(list_of_files, key=os.path.getctime)
    print(f"Playing: {latest_file}")
    latest_file = "runs_smw/videos/Dec22A/Dec22A_73_596.54.mp4"
    print(f"Playing: {latest_file}")
    display(Video(latest_file, embed=True, width=768))
else:
    print("No videos found yet.")
    
video = "./runs_smw/videos/Dec22A/Dec22A_73_596.54.mp4"
# display(Video(video, embed=True, width=768))

NameError: name 'os' is not defined

In [ ]:
import cv2

cap = cv2.VideoCapture("runs_smw/videos/test_16.mp4")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    cv2.imshow("Frame-by-Frame", frame)

    # 關鍵：這裡等待按鍵。按 'n' 鍵跳到下一幀，按 'q' 離開
    key = cv2.waitKey(0) 
    if key == ord('q'):
        break
    elif key == ord('n'):
        continue

cap.release()
cv2.destroyAllWindows()